In [11]:
import os
import re
import sys
import json
import tweepy
import requests
import pandas as pd
import tweepy
from datetime import date

In [101]:

def get_keys(path):
    with open(path) as f:
        return json.load(f)
keys = get_keys("keys.txt")

today = date.today()
yesterday = "2022-04-20"

In [102]:
df_wapo = pd.read_csv("https://raw.githubusercontent.com/ndmvisuals/job-search-tool/main/data/thewashingtonpost-jobs.csv")
df_wapo['date_posted'] = pd.to_datetime(df_wapo['date_posted'],format = "%Y-%m-%d" )


In [103]:
def _summarize_job_time(df_date):
     df_job_time = df_date["job_duration"].value_counts().to_frame()
     df_job_time.reset_index(inplace=True)
     df_job_time = df_job_time.rename(columns = {'index':'job_type', "job_duration":"number_jobs"})
     return(df_job_time)

def _create_job_sentence(df_job_time):
    job_duration_sentences = []
    number_job_types = df_job_time.shape[0]

    for i in range(df_job_time.shape[0]):
        job_type = df_job_time.loc[i]["job_type"].lower()
        job_numbers = df_job_time.loc[i]["number_jobs"]
        job_type_sentence = f"{job_numbers} {job_type} jobs"
        if number_job_types >1 :
            if i+1 == number_job_types :
                job_duration_sentences.append(" and ")
                job_duration_sentences.append(job_type_sentence)
            
            else:
                if number_job_types >2:
                    job_duration_sentences.append(f"{job_type_sentence}, ")
                else:
                    job_duration_sentences.append(f"{job_type_sentence}")

        else:        
            job_duration_sentences.append(job_type_sentence) 

    sentence = "".join(job_duration_sentences)
    return(sentence)
    


def construct_tweet(company, df, date, day_word):
    # Filter jobs to specific date ---
    df_date = (df['date_posted'] >= date) 
    df_date = df.loc[df_date]
    # Summary jobs by job type
    df_job_time = _summarize_job_time(df_date)

    # Create job type summary sentences
    job_time_sentence = _create_job_sentence(df_job_time)
    main_tweet = f"{day_word}, {company} posted {job_time_sentence}. Job titles and links posted in thread: "

    # Create a thread for each job

    return(main_tweet)
   

In [104]:
main_tweet_message = construct_tweet("The Washington Post", df_wapo, "2022-04-22", "Yesterday")

In [121]:
client = tweepy.Client(consumer_key= keys["consumer_key"],
                       consumer_secret= keys["consumer_secret"],
                       access_token= keys["access_token"],
                       access_token_secret= keys["access_token_secret"],
                       bearer_token= keys["bearer_token"])

In [ ]:
auth = tweepy.OAuth1UserHandler(
   keys["consumer_key"], keys["consumer_secret"], keys["access_token"], keys["access_token_secret"]
)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

In [ ]:
api.user_timeline(screen_name = "JobJournalism")

In [106]:
response = client.create_tweet(text= main_tweet_message)

In [ ]:
client.get_user(username= "ndm_visuals")

In [127]:
client.get_list_tweets("1517697407626194944")

Unauthorized: 401 Unauthorized

In [110]:
thread = client.create_tweet(text = "testing thread", in_reply_to_tweet_id= "1517899548898779136" )